In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

Consolidate data

In [ ]:
# Load all CSV files from a folder
folder_path = "data"  # Change this to your actual folder path
all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".csv")]
all_files.sort()

df_list = [pd.read_csv(file) for file in all_files]
df = pd.concat(df_list, ignore_index=True)

# Ensure datetime column is in datetime format
df["Datetime"] = pd.to_datetime(df["Datetime"])

# Sort
df = df.sort_values(by='Datetime').reset_index(drop=True)

# Replace temperatures > 50°C with NaN. They are errors
df.loc[df['Temperature'] > 50, 'Temperature'] = np.nan

# Save to CSV
file_name = "data_consolidated.csv"
df.to_csv(file_name, index=False)

7 day moving average representation

In [ ]:
# Read from CSV
file_name = "data_consolidated.csv"
df = pd.read_csv(file_name, parse_dates=['Datetime'])

def resample_to_hourly(df):
    df_hourly = df.set_index('Datetime').groupby(pd.Grouper(freq='1h')).agg({
        'Temperature': 'mean',
        'Dew Point': 'mean',
        'Humidity': 'mean',
        'Wind Speed': 'mean',
        'Wind Gust': 'mean',
        'Pressure': 'mean',
        'Precip.': 'sum',
        'Wind': 'first',
        'Condition': 'first'
    }).reset_index()
    
    return df_hourly

# We will resample hourly, as different stations have different updating frequencies
df = resample_to_hourly(df)

In [ ]:
# Rolling 7 day average

df = df.set_index('Datetime')

numeric_cols = ['Temperature', 'Dew Point', 'Humidity', 'Wind Speed', 'Wind Gust', 'Pressure', 'Precip.']
non_numeric_cols = ['Wind', 'Condition']

# Rolling mean on numeric columns
df_numeric = df[numeric_cols].rolling('7D', center=True).mean()

# Group non-numeric columns with Grouper + transform('first')
df_non_numeric = df[non_numeric_cols].copy()
df_non_numeric = df_non_numeric.groupby(pd.Grouper(freq='1h')).transform(lambda x: x)

# Combine both
df_result = pd.concat([df_numeric, df_non_numeric], axis=1).reset_index()

In [ ]:
# Extract year and set X-axis as "day of the year" (keeping hours & minutes for full detail)
df_result["year"] = df_result["Datetime"].dt.year.astype(str)  # Convert year to string for Plotly
df_result["day_of_year"] = df_result["Datetime"].apply(lambda x: x.replace(year=2000))  # Normalize all years to 2000 for correct ordering

In [ ]:
def plot_hourly_climatology(df, value_col, y_label="Value", recent_years=11):
    df['hour_of_year'] = df['Datetime'].dt.dayofyear + (df['Datetime'].dt.hour / 24)

    unique_years = sorted(df['year'].unique())
    highlight_years = unique_years[-recent_years:]
    latest_year_str = str(highlight_years[-1])
    old_years_label = f"{df['year'].min()}-{unique_years[-recent_years-1]}"

    # Step 1: Create an empty fig and add the dummy trace first
    fig = px.line(df.iloc[0:1], x="hour_of_year", y=value_col)  # Empty base fig
    fig.add_scatter(
        x=[None], y=[None], mode='lines',
        line=dict(color='#BBBBBB', width=1),
        name=old_years_label
    )

    # Step 2: Create actual data plot separately
    fig_data = px.line(
        df,
        x="hour_of_year",
        y=value_col,
        color="year",
        line_group="year",
        labels={value_col: y_label, "hour_of_year": "Day of Year"},
        render_mode="webgl"
    )

    # Step 3: Append actual traces to the base fig
    for trace in fig_data.data:
        year = trace.name.strip()
        if year == latest_year_str:
            trace.line.width = 3
            trace.line.color = 'red'
            fig.add_trace(trace)
        elif year in [str(y) for y in highlight_years]:
            trace.line.width = 1
            fig.add_trace(trace)
        else:
            trace.line.color = '#BBBBBB'
            trace.line.width = 1
            trace.showlegend = False
            fig.add_trace(trace)

    # Step 4: Axis formatting
    month_starts = pd.date_range('2020-01-01', '2020-12-01', freq='MS')
    tickvals = month_starts.dayofyear.tolist()
    ticktext = month_starts.strftime('%b').tolist()

    title = f"7 Day moving average {value_col}. Zaragoza"
    
    fig.update_layout(
        width=1000,
        height=600,
        xaxis_title="Month",
        yaxis_title=y_label,
        title = title,
        xaxis=dict(
            tickmode='array',
            tickvals=tickvals,
            ticktext=ticktext,
            tickangle=0,
            range=[1, 366],
            showgrid=True,
            gridcolor='white',
            gridwidth=1,
            tickson="boundaries",
            showline=True
        )
    )

    fig.show()


In [ ]:
plot_hourly_climatology(df_result, value_col="Temperature", y_label="Temperature (°C)")
plot_hourly_climatology(df_result, value_col="Humidity", y_label="Relative Humidity (%)")
plot_hourly_climatology(df_result, value_col="Wind Speed", y_label="Wind Speed (km/h)")

Percentile representation

In [ ]:
# Ensure datetime column is parsed correctly
df_result['year'] = df_result['Datetime'].dt.year
df_result['dayofyear'] = df_result['Datetime'].dt.dayofyear
df_result['hour'] = df_result['Datetime'].dt.hour

lastyear = df_result['year'].max()
firstyear = df_result['year'].min()

# Filter for historical years
df_hist = df_result[df_result['year'] < lastyear]

# Group by day of year and hour
agg_df = df_hist.groupby(['dayofyear', 'hour'])['Temperature'].agg([
    ('avg_temperature', lambda x: np.nanmean(x)),
    ('percentile_10', lambda x: np.percentile(x.dropna(), 10) if x.dropna().size > 0 else np.nan),
    ('percentile_90', lambda x: np.percentile(x.dropna(), 90) if x.dropna().size > 0 else np.nan),
    ('percentile_25', lambda x: np.percentile(x.dropna(), 25) if x.dropna().size > 0 else np.nan),
    ('percentile_75', lambda x: np.percentile(x.dropna(), 75) if x.dropna().size > 0 else np.nan),
    ('min_temperature', lambda x: np.nanmin(x)),
    ('max_temperature', lambda x: np.nanmax(x))
]).reset_index()

# Optional: add a "Datetime" reference without year for easier plotting
agg_df['reference_date'] = pd.to_datetime('2024-01-01') + pd.to_timedelta(agg_df['dayofyear'] - 1, unit='d') + pd.to_timedelta(agg_df['hour'], unit='h')

# Optional: sort by reference date
agg_df = agg_df.sort_values(['dayofyear', 'hour'])

# Filter current year data
df_lastyear['dayofyear'] = df_lastyear['Datetime'].dt.dayofyear
df_lastyear['hour'] = df_lastyear['Datetime'].dt.hour
df_lastyear['reference_date'] = pd.to_datetime('2024-01-01') + pd.to_timedelta(df_lastyear['dayofyear'] - 1, unit='d') + \
        pd.to_timedelta(df_lastyear['hour'], unit='h')

# Sort both dataframes by reference date
agg_df = agg_df.sort_values('reference_date')
df_lastyear = df_lastyear.sort_values('reference_date')

In [ ]:
# Create the figure
fig = go.Figure()

# Light gray band: min to max
fig.add_traces(go.Scatter(
    x=agg_df['reference_date'],
    y=agg_df['max_temperature'],
    mode='lines',
    line=dict(color='lightgray', width=0),
    name='Max',
    showlegend=False
))
fig.add_traces(go.Scatter(
    x=agg_df['reference_date'],
    y=agg_df['min_temperature'],
    mode='lines',
    line=dict(color="#CCCCCC", width=0),
    fill='tonexty',
    fillcolor='rgba(221,221,221,1)',  # Light gray with transparency
    name='Min-Max range',
))

# Gray band: 10th to 90th percentile
fig.add_traces(go.Scatter(
    x=agg_df['reference_date'],
    y=agg_df['percentile_90'],
    mode='lines',
    line=dict(color="#AAAAAA", width=0),
    name='80th percentile',
    showlegend=False
))
fig.add_traces(go.Scatter(
    x=agg_df['reference_date'],
    y=agg_df['percentile_10'],
    mode='lines',
    line=dict(color="#AAAAAA", width=0),
    fill='tonexty',
    fillcolor='rgba(170,170,170,1)',  # Darker gray with transparency
    name='10-90 percentile range',
))

# Dark gray band: 25th to 75th percentile
fig.add_traces(go.Scatter(
    x=agg_df['reference_date'],
    y=agg_df['percentile_75'],
    mode='lines',
    line=dict(color="#888888", width=0),
    name='80th percentile',
    showlegend=False
))
fig.add_traces(go.Scatter(
    x=agg_df['reference_date'],
    y=agg_df['percentile_25'],
    mode='lines',
    line=dict(color="#888888", width=0),
    fill='tonexty',
    fillcolor='rgba(136,136,136,1)',  # Darker gray with transparency
    name='25-75 percentile range',
))

# Black average line
fig.add_trace(go.Scatter(
    x=agg_df['reference_date'],
    y=agg_df['avg_temperature'],
    mode='lines',
    line=dict(color="#000000", width=2),
    name='Historical Avg'
))

# Red line for last year
fig.add_trace(go.Scatter(
    x=df_lastyear['reference_date'],
    y=df_lastyear['Temperature'],
    mode='lines',
    line=dict(color='red', width=2),
    name=f'{lastyear} Temperature'
))

# Layout tweaks
fig.update_layout(
    title=f"7 Day moving average Temperature Comparison Zaragoza (Historical {firstyear}-{lastyear-1} vs {lastyear})",
    xaxis_title="Date",
    yaxis_title="Temperature (°C)",
    template='plotly_white',
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    xaxis=dict(
        tickformat="%b",  # Only month abbreviation, like "Jan"
        dtick="M1",       # Force a tick every month
        ticklabelmode="period"  # Makes sure ticks appear at the start of each month
    )
)

fig.show()